# <span style='color:orange'> Load Model and Apply on Validation Data</span>

## Import modules

In [1]:
import h5py
import sys
sys.path.append('..')
from modules.configfile import config
from modules.training_helpers import *
import cPickle as pickle
import numpy as np
import random as random
random.seed(1337)
np.random.seed(1337)
import importlib
import optparse
import os
import logging
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

logging.basicConfig(level=logging.DEBUG)
try:
    logger = logging.getLogger(__file__.split('/')[-1])
except:
    logger = logging.getLogger(__name__)


Using TensorFlow backend.


### Create a custom class to mimic the "options" data structure, since optparse doesn't work in Jupyter

In [2]:
class OPT:
    def __init__(self):
        self.defmodelfile = 'cnn_patches'
        self.grade = 'HGG'
        self.output_name = 'cnn_patches_v1.h5'

In [3]:
options = OPT()

In [4]:
if options.output_name is None:
    logger.info('No output name defined, using default values')
    options.output_name = os.path.join(config['model_snapshot_location'], 'model_default.h5')
    logger.info('Name of output file: {}'.format(options.output_name))
else:
    options.output_name = os.path.join(config['model_snapshot_location'], options.output_name)
    logger.info('Name of output file: {}'.format(options.output_name))

if options.defmodelfile is None:
    logger.info('No defmodel file name defined, using default model (cnn)')
    options.defmodelfile = 'cnn'

INFO:__main__:Name of output file: /local-scratch/cedar-rm/scratch/asa224/model-snapshots/cnn_patches_v1.h5


In [5]:
options.output_name

'/local-scratch/cedar-rm/scratch/asa224/model-snapshots/cnn_patches_v1.h5'

## Get the module name to load, and open mean_var file

In [6]:
# --------------------------------------------------------------------------------------
# open required files
# --------------------------------------------------------------------------------------
# open mean and variance dictionary

mean_var = pickle.load(open(config['saveMeanVarFilepath'+options.grade.upper()], 'rb'))

# open new database with cropped images

if config['validate_on'] == 'cropped':
    hdf5_file = h5py.File(config['hdf5_filepath_cropped'], mode='r')
    hdf5_file_g = hdf5_file['preprocessed']
else:
    hdf5_file = h5py.File(config['hdf5_filepath_prefix'], mode='r')
    hdf5_file_g = hdf5_file['original_data']

# get the validation data
# training_data = hdf5_file_g['training_data_'+options.grade.lower()]
# training_data_segmasks = hdf5_file_g['training_data_segmasks_'+options.grade.lower()]

if options.defmodelfile is None:
    logger.info('No defmodel file name defined, using default model (cnn)')
    options.defmodelfile = 'cnn'
    
# -------------------------------------------------------------------------------------
# get model file
# -------------------------------------------------------------------------------------
# open the model file module
modeldefmodule = importlib.import_module('defmodel.'+options.defmodelfile, package=None)

INFO:cnn_patches.pyc:Setting keras backend data format to "channels_first"


## Load the model, hyperparameters and history

In [7]:
model, params, history = modeldefmodule.open_model_with_hyper_and_history(name=options.output_name, custom_obj={'dice_coefficient_loss':modeldefmodule.dice_coefficient_loss})

INFO:cnn_patches.pyc:Opening trained model with name /local-scratch/cedar-rm/scratch/asa224/model-snapshots/cnn_patches_v1.h5
INFO:cnn_patches.pyc:Model open successful!
INFO:cnn_patches.pyc:Opening hyperparameter dictionary with name /local-scratch/cedar-rm/scratch/asa224/model-snapshots/cnn_patches_v1_hyper_dict.p
INFO:cnn_patches.pyc:Opened hyperparameter dictionary!
INFO:cnn_patches.pyc:Opening history object with name /local-scratch/cedar-rm/scratch/asa224/model-snapshots/cnn_patches_v1_hyper_dict.p
INFO:cnn_patches.pyc:Opened history object!


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 4, None, None, Non 0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 32, None, None, No 3488      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 32, None, None, No 27680     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 64, None, None, No 55360     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 4, None, None, Non 6916      
Total params: 93,444
Trainable params: 93,444
Non-trainable params: 0
_________________________________________________________________


## Generate synthetic data of various sizes

In [9]:
# one patient, with 4 modalities, of original size
pat1 = np.empty((1, 4, 120, 120, 120))

## Standardize the data with mean and var values

In [10]:
from modules.dataloader import standardize

In [11]:
pat1.shape

(1, 4, 120, 120, 120)

In [12]:
pat1_s = standardize(pat1, applyToTest=mean_var)

INFO:modules.dataloader:Applying to test data using provided values


## Push the data to model

In [13]:
pred = model.predict(pat1)

In [14]:
pred.shape

(1, 4, 120, 120, 120)